*<font size = 5> Онкология. Заболеваемость и смертность в разрезе пола и возраста. РФ, 2021 г </font>*


**Автор:** Александр Куроедов

**Исходные данные:** Ежегодный отчет главного онколога РФ опубликованного за 2021 год. Таблицы 9-11 и 66-68.

**Ссылка на готовый дашборд:** https://datalens.yandex/dqveeiogswx82

### Подготовка данных

In [1]:
# подключим библиотеки
import numpy as np 
import pandas as pd
import csv
# настроим вывод
pd.set_option('display.max_columns', None)
# отключим предупреждения
import warnings
warnings.filterwarnings('ignore')

In [2]:
folder = 'C:/Users/aleks/Desktop/DA обучение/PET/drive-download-20231018T173209Z-001/'
# составим список таблиц и укажем их расположение
table1 = folder+'2021_Таблица_009_Злокачественные_новообразования_в_РФ_(заболеваемость_и_смертность).xlsx'
table2 = folder+'2021_Таблица_010_Злокачественные_новообразования_в_РФ_(заболеваемость_и_смертность).xlsx'
table3 = folder+'2021_Таблица_011_Злокачественные_новообразования_в_РФ_(заболеваемость_и_смертность).xlsx'
table4 = folder+'2021_Таблица_066_Злокачественные_новообразования_в_РФ_(заболеваемость_и_смертность).xlsx'
table5 = folder+'2021_Таблица_067_Злокачественные_новообразования_в_РФ_(заболеваемость_и_смертность).xlsx'
table6 = folder+'2021_Таблица_068_Злокачественные_новообразования_в_РФ_(заболеваемость_и_смертность).xlsx'


In [3]:
def transformation(table, result_stats, gender):
    # считываем 
    df = pd.read_excel(table, header=[0,1,2,3,4])
    df = df.droplevel([0,1,2,3], axis=1).rename(columns={
        'Unnamed: 0_level_4':'Локализация, нозологическая форма',
        'Unnamed: 1_level_4':'to_drop1',
        'Unnamed: 2_level_4':'Код МКБ 10',
        'Unnamed: 3_level_4':'Станд. показатель',
        'Unnamed: 4_level_4':'Ошиб. станд. показ.',
        'Unnamed: 5_level_4':'Показатель'
    }
    ).drop(columns=['to_drop1'])
    # обрабатываем
    df = df.drop(columns=df.iloc[:,4:]).merge(
        df.iloc[:,5:].stack().reset_index().drop(columns=[0]).merge(
            df.iloc[:,5:].stack().reset_index().iloc[:,2].str.split('\n', expand=True),
            left_index=True, right_index=True
        ).rename(columns={'level_0':'Index','level_1':'Возраст',0:'Абс.число',1:'"Грубый"',2:'Уд.вес,%'}),
        left_index=True, right_on='Index'
    ).drop(columns=['Index'])
    df['Код МКБ 10'] = df['Код МКБ 10'].str.replace('\n',' ')
    df['Пол'] = gender
    df['Статистика'] = result_stats
    return df

In [4]:
# вызываем и прогружаем. Собираем единую таблицу.
final_table =\
transformation(table1, 'заболеваемость', 'оба').append(transformation(table2, 'заболеваемость', 'мужской'), sort=False).\
append(transformation(table3, 'заболеваемость', 'женский'), sort=False).\
append(transformation(table4, 'смертность', 'оба'), sort=False).\
append(transformation(table5, 'смертность', 'мужской'), sort=False).\
append(transformation(table6, 'смертность', 'женский'), sort=False).reset_index().drop(columns='index')

In [5]:
# посмотрим на таблицу
final_table.head(5)

,"Локализация, нозологическая форма",Код МКБ 10,Станд. показатель,Ошиб. станд. показ.,Возраст,Абс.число,"""Грубый""","Уд.вес,%",Пол,Статистика
0,Злокачественные новообразования - всего,С00-96,224.87,0.32,Всего,580415,"397,91","100,00",оба,заболеваемость
1,Злокачественные новообразования - всего,С00-96,224.87,0.32,0-4,1343,"17,13","100,00",оба,заболеваемость
2,Злокачественные новообразования - всего,С00-96,224.87,0.32,5-9,871,"9,08","100,00",оба,заболеваемость
3,Злокачественные новообразования - всего,С00-96,224.87,0.32,10-14,784,"9,30","100,00",оба,заболеваемость
4,Злокачественные новообразования - всего,С00-96,224.87,0.32,15-19,1223,"16,63","100,00",оба,заболеваемость


In [6]:
# проверим типы данных
final_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4104 entries, 0 to 4103
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Локализация, нозологическая форма  4104 non-null   object 
 1   Код МКБ 10                         4104 non-null   object 
 2   Станд. показатель                  4104 non-null   float64
 3   Ошиб. станд. показ.                4104 non-null   float64
 4   Возраст                            4104 non-null   object 
 5   Абс.число                          4104 non-null   object 
 6   "Грубый"                           4104 non-null   object 
 7   Уд.вес,%                           4104 non-null   object 
 8   Пол                                4104 non-null   object 
 9   Статистика                         4104 non-null   object 
dtypes: float64(2), object(8)
memory usage: 320.8+ KB


In [7]:
# преобразуем типы данных в соответствующие
final_table['Абс.число'] = final_table['Абс.число'].astype('int')
final_table['"Грубый"'] = final_table['"Грубый"'].str.replace(',','.').astype('float')
final_table['Уд.вес,%'] = final_table['Уд.вес,%'].str.replace(',','.').astype('float')

In [8]:
# контроль
final_table.tail(5)

,"Локализация, нозологическая форма",Код МКБ 10,Станд. показатель,Ошиб. станд. показ.,Возраст,Абс.число,"""Грубый""","Уд.вес,%",Пол,Статистика
4099,Лимфатическая и кроветворная ткань,С81-96,4.27,0.06,65-69,1076,20.54,5.25,женский,смертность
4100,Лимфатическая и кроветворная ткань,С81-96,4.27,0.06,70-74,1161,28.27,5.47,женский,смертность
4101,Лимфатическая и кроветворная ткань,С81-96,4.27,0.06,75-79,677,36.33,5.76,женский,смертность
4102,Лимфатическая и кроветворная ткань,С81-96,4.27,0.06,80-84,903,34.47,4.81,женский,смертность
4103,Лимфатическая и кроветворная ткань,С81-96,4.27,0.06,85-,424,25.84,3.40,женский,смертность


In [9]:
# сохраняем в файл
final_table.to_csv("medical_dashboard_group_1.csv", sep='\t', index=False)

### Расчет перцентилей для визуализации показателей.

Возьмем следующие показатели для цветовой визуализации в сводной таблице:
- 50 - "типовой"
- 75 - "вызывает настороженность"
- 90 - "особое внимание"

В будущем следует согласовать эти показатели с заказчиком.

In [10]:
# сначала фильтруем избыточную информацию (показатели "всего")
temp = final_table.query('`Локализация, нозологическая форма` != "Злокачественные новообразования - всего" & Возраст != "Всего"')

# рассчитаем перцентили для визуализации по показателям
# Абс.число
np.percentile(temp[['Абс.число']], [50, 75, 90])

array([  58. ,  291. , 1136.7])

In [11]:
# "Грубый"
np.percentile(temp[['"Грубый"']], [50, 75, 90])

array([ 1.26  ,  7.2825, 28.515 ])

In [12]:
# Уд.вес,%
np.percentile(temp[['Уд.вес,%']], [50, 75, 90])

array([0.86 , 3.38 , 7.924])

### Краткие выводы по результату создания дашборда.


По результат обработки исходных .xlsx файлов были умышленно сохранены:
- строка "Злокачественные новообразования - всего"
- столбец "Возраст - Всего"

т.к. в процессе построения дашборда выявлены некоторые несоответствия цифр. Например в таблице 010 столбец "Возраст" - "50-54" общая цифра абсолютного значения 15629, однако при вычислении получается иной результат. 

In [13]:
final_table.query('Возраст=="50-54" &\
                  `Локализация, нозологическая форма`!="Злокачественные новообразования - всего" &\
                  Пол=="мужской" &\
                  Статистика=="заболеваемость"')['Абс.число'].sum()

15996

Поэтому было принято решение использовать уже готовые итоговые значения. 


В рамках имеющейся задачи по визуализации было принято совместить графическое отображение данных с более консервативным (и вероятно более привычным представителям медицины) вариантом в виде сводной таблицы.

В итоге я использовал столбчатую диаграмму для визуализации заболеваемости и смертности в разрезе возраста по абсолютным данным, а также таблицу с сортировкой по самым часто встречающимся локализациям онкопроцесса на основе стандартизированных показателей. 
Пол для этих визуализаций выбирается интерактивно.

Вторая часть визуализации - сводная таблица - сделана в виде хитмэпа. Она может строиться на основе грубых, стандартизированных или абсолютных значений и позволяет интерактивно выбирать параметры фильтрации данных, а именно:
- пол
- статистику по смертности/заболеваемости
- возраст.

Кроме того, таблица имеет пояснения по цветовому отображению данных. В будущем следует уточнить у представителя медицины, какие перцентили лучше использовать.

Я не являюсь представителем медицинского сообщества, поэтому мне сложно выдвигать какие либо гипотезы, однако по созданному дашборду могу сделать, например, следующие выводы и предположения:
- после 30 лет идет рост количества онкозаболеваний у обоих полов
- пик заболеваемости среди как мужского, так и женского населения приходится на 65-69 лет
- у женской части населения также есть повторный подъём заболеваемости в 80-84 года
- локализация наибольшей части онкозаболеваний у мужчин - трахея, бронхи, легкое, возможно причина этому неблагоприятные условия труда и курение. У женской части населения это - молочная железа, очевидно здесь взаимосвязь с особенностями строения женского организма.
-  детском возрасте высока вероятность поражения головного мозга, ЦНС, лимфатической и кроветворной ткани, а в среднем и пожилом - трахеи, легкого, ободочной кишки и кожи.